<a href="https://colab.research.google.com/github/enofitriyanti11/LSTM-MI3A-Kelompok-1/blob/main/TUBES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#1. Preparation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("/content/sample_data/tubes.csv")
df.head()


,text,label
0,Berantas dan usut tuntas kriminalisasi Hanfah ...,positif
1,RT Ceeek berita ini fakta kriminalisasi Hanifa...,negatif
2,RT Tolong dong masa instansi yang melindungi r...,positif
3,"Miris bgt deh. Kasian bu hanifah husein, istri...",negatif
4,RT Duh ga nyangka sih kalo ternyata Bu Hanifa...,positif


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    699 non-null    object
 1   label   699 non-null    object
dtypes: object(2)
memory usage: 11.0+ KB


In [ ]:
df.label.value_counts()

positif    354
netral     178
negatif    167
Name: label, dtype: int64

#2. Data Cleaning, Splitting dan Pre-Processing

In [ ]:
from bs4 import BeautifulSoup
import re
import nltk
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')

pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
gabung_pat = r'|'.join((pat1, pat2))
stopword = set(stopwords.words('english'))

def tweet_cleaner(text):
  soup = BeautifulSoup(text, "html.parser")
  souped = soup.get_text()
  stripped = re.sub(gabung_pat, '', souped)
  try:
    clean = stripped.decode("utf-8").replace(u"\ufffd", "?")
  except:
    clean = stripped
  letter_only = re.sub("[^a-zA-Z]", " ", clean)
  lower_case = letter_only.lower()
  words = token.tokenize(lower_case)
  filtered_word = [w for w in words if not w in stopword]
  return(" ".join(filtered_word)).strip() 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
teks = df.text[:700]
test_result = []
for t in teks:
  test_result.append(tweet_cleaner(t))

In [ ]:
test_result

['berantas dan usut tuntas kriminalisasi hanfah husein',
 'rt ceeek berita ini fakta kriminalisasi hanifah husein',
 'rt tolong dong masa instansi yang melindungi rakyat bisa begini',
 'miris bgt deh kasian bu hanifah husein istri mantan menteri atr bpn ferry mursyidan baldan suami baru meninggal lahan tambang diambil org lain secara serampangan trus jadi korban kriminalisasi',
 'rt duh ga nyangka sih kalo ternyata bu hanifah husein di kriminalisasi semoga segera diusut tuntas ini kasusnya',
 'usut tuntass',
 'rt kurang ajar masih aja suka cari kesalahan bukti nyata kriminalisasi hanifah husein hanifah husein',
 'rt tindak tegas siapapun itu dalam tindakan diskriminasi sekalipun itu di institusi polri',
 'kasian banget bu hanifah husein istri mantan menteri atr bpn ferry mursyidan baldan suami baru meninggal lahan tambang diambil org lain secara serampangan trus jadi korban kriminalisasi',
 'trending indonesia',
 'rt sumpah sedih banget baru tau kalo hu hanifah husein kriminalisasi',
 

In [ ]:
%%time
print("Cleaning and Parsing Tweets...\n")
#teks = df.text[:500]
clean_tweet_text = []
for i in teks:
  clean_tweet_text.append(tweet_cleaner(i))
print("Done!")

Cleaning and Parsing Tweets...

Done!
CPU times: user 50.2 ms, sys: 1.53 ms, total: 51.8 ms
Wall time: 56.4 ms


In [ ]:
clean_df = pd.DataFrame(clean_tweet_text,columns=['Text'])
clean_df['Label'] = df.label
clean_df.head()

,Text,Label
0,berantas dan usut tuntas kriminalisasi hanfah ...,positif
1,rt ceeek berita ini fakta kriminalisasi hanifa...,negatif
2,rt tolong dong masa instansi yang melindungi r...,positif
3,miris bgt deh kasian bu hanifah husein istri m...,negatif
4,rt duh ga nyangka sih kalo ternyata bu hanifah...,positif


In [ ]:
clean_df['Label'] = clean_df['Label'].map({'negatif': 0, 'positif': 1, 'netral': 2})
clean_df.head()

,Text,Label
0,berantas dan usut tuntas kriminalisasi hanfah ...,1
1,rt ceeek berita ini fakta kriminalisasi hanifa...,0
2,rt tolong dong masa instansi yang melindungi r...,1
3,miris bgt deh kasian bu hanifah husein istri m...,0
4,rt duh ga nyangka sih kalo ternyata bu hanifah...,1


Split Training dan Testing Data

In [ ]:
from sklearn.model_selection import train_test_split

X = clean_df['Text'].values
#clean_df = clean_df[clean_df.Label != 2]
y = clean_df['Label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30,  random_state=42)

#3. Feature Extraction

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()
X_train = vect.fit_transform(X_train)
print("Number of features:  %d" % len(vect.vocabulary_))
X_test = vect.transform(X_test)

Number of features:  865


In [ ]:
from sklearn import svm
from sklearn.metrics import accuracy_score

classify = svm.SVC(kernel='linear')
print("Training Classifier...")
classify.fit(X_train, y_train)
print("Predicting...")
prediksi = classify.predict(X_test)
akurasi = accuracy_score(y_test, prediksi)
print('Accuracy:', akurasi)

Training Classifier...
Predicting...
Accuracy: 0.6333333333333333


In [ ]:
print("Number of features:  %d" % len(vect.vocabulary_))
print("------------------")

sum_words = X_train.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in vect.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
most_freq = words_freq[:10]
for word, freq in most_freq:
  print(word, freq)

#X_test = vectorizer.transform(X_test)
print("------------------")
print("Training Classifier...")
classify.fit(X_train, y_train)
print("Predicting...")
prediksi = classify.predict(X_test)
akurasi = accuracy_score(y_test, prediksi)
print ('Accuracy:', akurasi)

Number of features:  865
------------------
kriminalisasi 275
hanifah 244
husein 220
ini 130
ada 113
di 111
kriminalisasihanifahhusein 99
tuntas 93
yang 77
usut 72
------------------
Training Classifier...
Predicting...
Accuracy: 0.6333333333333333


In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

print("Number of features:  %d" % len(vect.vocabulary_))
print ("---------------------")

#print features their chi2 score
fitur_skor = chi2(X_train, y_train)[0]
for score, fname in sorted(zip(fitur_skor, vect.get_feature_names()), reverse=True)[:10]:
    print(fname, score)

#selectKBest feature using Chi2 and see whether it could improve the accuracy
ch2 = SelectKBest(chi2, k=865)
X_train_best = ch2.fit_transform(X_train, y_train)
X_test_best = ch2.transform(X_test)
print ("---------------------")
print("Training Classifier...")
classify.fit(X_train_best, y_train)
print("Predicting...")
prediksi = classify.predict(X_test_best)
akurasi = accuracy_score(y_test, prediksi)
print ('Accuracy:', akurasi)

Number of features:  865
---------------------
ceeek 41.33333333333333
fakta 22.92194453460276
hukum 21.056939696745786
safeflightourastronaut 19.546329113924052
kriminalisasihanifahhusein 19.349261084282176
rt 18.084745521707546
berita 17.43103270519503
semoga 17.296114138128683
meninggal 17.222412155745488
diambil 17.222412155745488
---------------------
Training Classifier...
Predicting...
Accuracy: 0.6333333333333333


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
from sklearn.feature_selection import mutual_info_classif

print("Number of features:  %d" % len(vect.vocabulary_))
print ("---------------------")

#print features their MI score
fitur_skor = mutual_info_classif(X_train, y_train)
for score, fname in sorted(zip(fitur_skor, vect.get_feature_names()), reverse=True)[:10]:
    print(fname, score)

#selectKBest feature using Mutual Information and see whether it could improve the accuracy
mic = SelectKBest(mutual_info_classif, k=865)
X_train_best = mic.fit_transform(X_train, y_train)
X_test_best = mic.transform(X_test)
print ("---------------------")
print("Training Classifier...")
classify.fit(X_train_best, y_train)
print("Predicting...")
prediksi = classify.predict(X_test_best)
akurasi = accuracy_score(y_test, prediksi)
print ('Accuracy:', akurasi)

Number of features:  865
---------------------


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


ceeek 0.039199717542785326
rt 0.026857656466190746
hukum 0.025369158736508864
kriminalisasihanifahhusein 0.02516976413477881
fakta 0.02426582392395457
tegakkan 0.02273438076966472
seadil 0.021185638084536634
adilnya 0.021185638084536634
semoga 0.02066758138934048
permainan 0.019641760543066406
---------------------
Training Classifier...
Predicting...
Accuracy: 0.6333333333333333
